# Wave Energy
Notebook environment to migrate netcdf files to CF compliant zarr

In [ ]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

### Configure OS independent paths

In [ ]:
# Import standard packages
import os
import pathlib
import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4 as nc
import numpy.ma as ma

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))

# Get root paths
home = pathlib.Path().home()
root = home.root

# Import custom functionality
from etl import p_drive
from etl.CF_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
coclico_data_dir = p_drive.joinpath("11205479-coclico", "data")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

In [ ]:
# Project paths & files (manual input)
ds_dir = coclico_data_dir.joinpath("02_wave_energy_jrc")
ds_rcp85_path = ds_dir.joinpath("CoastAlRisk_Global_WEF_RCP85.nc")
ds_out_file = "CoastAlRisk_Global_WEF_RCP85"
CF_dir = coclico_data_dir.joinpath(r"CF")  # directory to save output CF check files

### Check CF compliancy original NetCDF files

In [ ]:
# open datasets
ds_85rcp = xr.open_dataset(ds_rcp85_path)

# check original dataset
ds_85rcp

In [ ]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=ds_rcp85_path, working_dir=CF_dir)

In [ ]:
# save original CF compliancy
save_compliancy(cap, testfile=ds_rcp85_path, working_dir=CF_dir)

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [ ]:
# NetCDF attribute alterations

# rename global attribute names
ds_85rcp.attrs["Project_Name"] = ds_85rcp.attrs.pop("Project Name")
ds_85rcp.attrs["Project_Acronym"] = ds_85rcp.attrs.pop("Project Acronym")

# add global attributes
ds_85rcp.attrs["Conventions"] = "CF-1.8"
ds_85rcp.attrs["Starting_date"] = "01-Dec-2009"
ds_85rcp.attrs["End_date"] = "30-Nov-2099 21:00:00"

# remove certain variable attributes
del ds_85rcp["rp"].attrs["Starting date"]
del ds_85rcp["rp"].attrs["End date"]

# add or change certain variable / coordinate attributes
dataset_attributes = {
    "decades": {"long_name": "decade window", "Format": "YYYY", "units": "yr"}
}  # specify custom (CF convention) attributes

# add / overwrite attributes
for k, v in dataset_attributes.items():
    try:
        ds_85rcp[k].attrs = dataset_attributes[k]
    except:
        continue

In [ ]:
# NetCDF variable and dimension alterations

# rename or swap dimension names, the latter in case the name already exists as coordinate
ds_85rcp = ds_85rcp.rename_dims({"npoints": "stations", "nsdec": "time"})
ds_85rcp = ds_85rcp.swap_dims({"nrp": "rp"})

# rename variables, if necessary
ds_85rcp = ds_85rcp.rename_vars(
    {"longitude": "lon", "latitude": "lat", "decades": "time"}
)

# set some data variables to coordinates to avoid duplication of dimensions in later stage
ds_85rcp = ds_85rcp.set_coords(["lon", "lat", "rp", "time"])

In [ ]:
# re-order shape of the data variables
ds_85rcp = ds_85rcp.transpose("stations", "rp", "time")

# add epsg
ds_85rcp.attrs["crs"] = 4326

In [ ]:
# check the xarray dataset, best practice is to have as many as possible bold dimensions (dimension == coordinate name).
# in this way, the Front-End can access the variable directly without having to index the variable first

ds_85rcp

In [ ]:
# save new .nc files
ds_85rcp.to_netcdf(path=ds_dir.joinpath(ds_out_file + "_CF.nc"))

### Check CF compliancy altered NetCDF files

In [ ]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=ds_dir.joinpath(ds_out_file + "_CF.nc"), working_dir=CF_dir)

In [ ]:
# save altered CF compliancy
save_compliancy(
    cap, testfile=ds_dir.joinpath(ds_out_file + "_CF.nc"), working_dir=CF_dir,
)

### write data to Zarr files

In [ ]:
# export to zarr in write mode (to overwrite if exists)
ds_85rcp.to_zarr(ds_dir.joinpath("%s.zarr" % ds_out_file), mode="w")